In [ ]:
# jap2010を読み込んでparquetにするcode｡実行不要

"""
テキストアーカイブ - 日本語ウェブコーパス 2010
http://www.s-yata.jp/corpus/nwc2010/texts/
を処理するcode｡

#URLは常時は空いていないので注意｡(畠山が持ってます)
wget http://www.s-yata.jp/corpus/nwc2010/texts/filelist
wget -i filelist
"""

In [4]:
import os

out_dir="data/jap2010_parquet"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [32]:
import lzma
import pandas as pd

import glob


00000123
00000060
00000112
00000120
00000175
00000085
00000191
00000047
00000093
00000036
00000062
00000161
00000184
00000165
00000080
00000155
00000097
00000021
00000082
00000065
00000183
00000146
00000169
00000035
00000074
00000159
00000103
00000173
00000145
00000077
00000022
00000024
00000153
00000124
00000054
00000148
00000151
00000049
00000073
00000143
00000064
00000182
00000058
00000039
00000099
00000102
00000135
00000032
00000136
00000055
00000067
00000163
00000157
00000033
00000178
00000092
00000013
00000050
00000096
00000072
00000037
00000106
00000109
00000185
00000001
00000122
00000179
00000086
00000057
00000129
00000127
00000066
00000147
00000084
00000079
00000149
00000107
00000052
00000189
00000046
00000113
00000017
00000034
00000164
00000048
00000174
00000154
00000029
00000014
00000030
00000111
00000061
00000181
00000187
00000140
00000031
00000134
00000043
00000025
00000188
00000138
00000166
00000162
00000144
00000128
00000180
00000098
00000117
00000069
00000101
00000095
0

In [33]:
count_threshold=3
max_lines=2000


task_name="00000001"
for xz_path in glob.glob("data/jap2010/*.xz"):
    task_name=xz_path.split("/")[-1].split(".")[0]
    print(task_name)
    path=f"data/jap2010/{task_name}.xz"
    # .xz ファイルを開いて内容をチャンクで読み込む
    with lzma.open(path, 'rt') as file:
        lines=file.readlines()

    new_line_count=0

    current_doc=[]
    cnt=0
    text_len=0
    text_list=[]
    for line in (lines):
        if line=="\n":
            new_line_count+=1
        else:
            new_line_count=0
        current_doc.append(line)
        if new_line_count>=count_threshold or len(current_doc)>max_lines:
            text="\n".join(current_doc)
            text=text.strip()
            text=text.replace("\n\n","\n")
            current_doc=[]
            new_line_count=0

            """
            if text!="":
                d=json.dumps({"text":text},ensure_ascii=False)
                with open(f"{out_dir}/{task_name}.jsonl","a") as f:
                    f.write(f"{d}\n")
            """
            if text!="":
                #print(len(text))
                text_list.append(text)
                text_len+=len(text)
                #break

        if text_len>10**8:
            df=pd.DataFrame(text_list,columns=["text"])#.reset_index()
            df.to_parquet(f"{out_dir}/{task_name}_{cnt}.parquet")
            cnt+=1
            text_list=[]
            text_len=0
            #if cnt>100:
            #    break

    df=pd.DataFrame(text_list,columns=["text"]).reset_index()
    df.to_parquet(f"{out_dir}/{task_name}_{cnt}.parquet")

00000123
00000060
00000112
00000120
00000175
00000085
00000191
00000047
00000093
00000036
00000062
00000161


In [16]:
import pandas as pd
df=pd.read_parquet("data/jap2010_parquet/00000001_0.parquet")
df

,index,text
0,0,コーチングでEQを高めるアプローチ法(1)\nブログをはじめよう\n野口嘉則 公式ブログ 「...
1,1,コロプラ 〜携帯を持って外に出かけよう〜 : 端末からのアクセスでブログが見えるときは\nコ...
2,2,デイトレードの結果を日々公開するブログ:【12月6日(月)のデイトレ注目銘柄】\nデイトレー...
3,3,裏ドラゴンボールマニア:ドラゴンボールてれびっこクイズ\n\n裏ドラゴンボールマニア:セルの...
4,4,裏ドラゴンボールマニア:DBの続編漫画がAF以外にもあった\n\n裏ドラゴンボールマニア:つ...
...,...,...
86,86,〜見習いクライマーの弁護士生活〜\n\n〜見習いクライマーの弁護士生活〜\n< 「大統領の陰...
87,87,噴水の上に眠るものの影ーめばBlogー : 住んでみたいところ\n噴水の上に眠るものの影ーめ...
88,88,「 つるちゃんちのキュッヒェ in ドイツ 」:夏時間について\n夏時間について\n昨日から...
89,89,山本みわ 議員活動日記\n山本みわ 議員活動日記\n**大好きなつくばの未来あなたとつくりた...


In [18]:
#huggingface-cli upload --repo-type=dataset hatakeyama-llm-team/japanese2010 .

^C


In [19]:
from datasets import load_dataset
#load test
dataset=load_dataset("hatakeyama-llm-team/japanese2010",
                     streaming=True,
                     split="train",
                     )

/home/setup/miniconda3/envs/ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


In [23]:
cnt=0
for data in dataset:
    print(data["text"][:10])
    cnt+=1
    if cnt>10:
        break

コーチングでEQを高
自己受容ができていな
うなもの」です
いつ
★検索したいキーワー
Iメッセージで自分の
★本来の自分に戻る「
★週末起業サラリーマ
★中小企業のサポータ
からの質問があり、気
前からうすうす気付い
★イプログダイレクト


728

In [21]:
with open("t.txt", "w") as f:
    f.write(data["text"])